In [2]:
import pandas as pd
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import sqlalchemy
import h5py

In [3]:
db_connection_string = 'sqlite:///./Resources/products.db'
engine  = sqlalchemy.create_engine(db_connection_string)

inspector = sqlalchemy.inspect(engine)
table_names = inspector.get_table_names()
print(table_names)

['HSFT_1_Day_Candles', 'HSFT_Info', 'Indicators', 'MSFT_1_Day_Candles', 'MSFT_1_Min_Candles', 'MSFT_Info', 'TSLA_1_Day_Candles', 'TSLA_1_Min_Candles', 'TSLA_Info']


In [4]:
indicators_df = pd.read_sql_table('Indicators', con=engine, index_col='Datetime')

---

## Prepare the data to be used on a neural network model

In [5]:
print(indicators_df.head())

                       Open    High      Low   Close  Volume  CDLDOJI  \
Datetime                                                                
2021-09-02 12:56:00  733.89  734.31  733.750  734.21   28796        0   
2021-09-02 12:57:00  734.29  734.64  734.219  734.41   34903        0   
2021-09-02 12:58:00  734.35  734.63  734.260  734.50   19890        0   
2021-09-02 12:59:00  734.60  734.77  734.510  734.56    9812      100   
2021-09-02 13:00:00  734.35  734.66  734.290  734.50   23069        0   

                     Trailing Stop  Trade Signal  Pct Change   Stop Loss  \
Datetime                                                                   
2021-09-02 12:56:00     732.988571           1.0    0.000463  732.988571   
2021-09-02 12:57:00     733.199031           1.0    0.000272  733.199031   
2021-09-02 12:58:00     733.309457           1.0    0.000123  733.309457   
2021-09-02 12:59:00     733.406281           1.0    0.000082  733.406281   
2021-09-02 13:00:00     733.3626

In [6]:
# Review the data types associated with the columns
indicators_df.dtypes

Open                   float64
High                   float64
Low                    float64
Close                  float64
Volume                   int64
CDLDOJI                  int64
Trailing Stop          float64
Trade Signal           float64
Pct Change             float64
Stop Loss              float64
CDL3WHITESOLDIERS        int64
CDLABANDONEDBABY         int64
CDLADVANCEBLOCK          int64
CDLBELTHOLD              int64
CDLBREAKAWAY             int64
CDLCLOSINGMARUBOZU       int64
CDLCONCEALBABYSWALL      int64
CDLCOUNTERATTACK         int64
CDLDARKCLOUDCOVER        int64
dtype: object

In [7]:
display(indicators_df['Trade Signal'].value_counts())

 1.0    11282
 0.0      843
-1.0      350
Name: Trade Signal, dtype: int64

### Step 5: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “IS_SUCCESSFUL”. The remaining columns should define the features dataset. 



### Step 3: Encode the dataset’s categorical variables using `OneHotEncoder`, and then place the encoded variables into a new DataFrame.

In [8]:
# Create a list of categorical variables 
categorical_variables = ['Trade Signal']

# Display the categorical variables list
display(categorical_variables)

['Trade Signal']

In [9]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)


In [10]:
# Encode the categorcal variables using OneHotEncoder
encoded_data = enc.fit_transform(indicators_df[categorical_variables])


In [11]:
# Create a DataFrame with the encoded variables
encoded_df = pd.DataFrame(
    encoded_data,
    columns = enc.get_feature_names(categorical_variables)
)
# Review the DataFrame
display(encoded_df.head(10))

,Trade Signal_-1.0,Trade Signal_0.0,Trade Signal_1.0
0,0.0,0.0,1.0
1,0.0,0.0,1.0
2,0.0,0.0,1.0
3,0.0,0.0,1.0
4,0.0,0.0,1.0
5,0.0,0.0,1.0
6,0.0,0.0,1.0
7,0.0,1.0,0.0
8,0.0,1.0,0.0
9,0.0,1.0,0.0


In [12]:
encoded_df.rename(columns={'Trade Signal_-1.0': 'Bearish', 'Trade Signal_0.0': 'None', 'Trade Signal_1.0':'Bullsih'}, inplace=True)
encoded_df.drop(columns='None', inplace=True)
print(encoded_df.head())

   Bearish  Bullsih
0      0.0      1.0
1      0.0      1.0
2      0.0      1.0
3      0.0      1.0
4      0.0      1.0


### Step 4: Add the original DataFrame’s numerical variables to the DataFrame containing the encoded variables.

> **Note** To complete this step, you will employ the Pandas `concat()` function that was introduced earlier in this course. 

In [13]:
# Define the target set y using the IS_SUCCESSFUL column
y = encoded_df.copy()

# Display a sample of y
display(y.head())

,Bearish,Bullsih
0,0.0,1.0
1,0.0,1.0
2,0.0,1.0
3,0.0,1.0
4,0.0,1.0


In [14]:
# Define features set X by selecting all columns but IS_SUCCESSFUL
X = indicators_df.drop(columns=['Trade Signal'])

# Review the features DataFrame
display(X.head())

,Open,High,Low,Close,Volume,CDLDOJI,Trailing Stop,Pct Change,Stop Loss,CDL3WHITESOLDIERS,CDLABANDONEDBABY,CDLADVANCEBLOCK,CDLBELTHOLD,CDLBREAKAWAY,CDLCLOSINGMARUBOZU,CDLCONCEALBABYSWALL,CDLCOUNTERATTACK,CDLDARKCLOUDCOVER
Datetime,,,,,,,,,,,,,,,,,,
2021-09-02 12:56:00,733.89,734.31,733.750,734.21,28796,0,732.988571,0.000463,732.988571,0,0,0,0,0,0,0,0,0
2021-09-02 12:57:00,734.29,734.64,734.219,734.41,34903,0,733.199031,0.000272,733.199031,0,0,0,0,0,0,0,0,0
2021-09-02 12:58:00,734.35,734.63,734.260,734.50,19890,0,733.309457,0.000123,733.309457,0,0,0,0,0,0,0,0,0
2021-09-02 12:59:00,734.60,734.77,734.510,734.56,9812,100,733.406281,0.000082,733.406281,0,0,0,0,0,0,0,0,0
2021-09-02 13:00:00,734.35,734.66,734.290,734.50,23069,0,733.362619,-0.000082,733.406281,0,0,0,0,0,0,0,0,0


### Step 6: Split the features and target sets into training and testing datasets.


In [15]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Step 7: Use scikit-learn's `StandardScaler` to scale the features data.

In [16]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


---

## Compile and Evaluate a Binary Classification Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [17]:
# Define the the number of inputs (features) to the model
number_input_features = len(list(X.columns))
# Review the number of features
number_input_features


18

In [18]:
# Define the number of neurons in the output layer
number_output_neurons = 2

In [19]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 =  int(round((number_input_features + number_output_neurons)/2, 0))
# Review the number hidden nodes in the first layer
hidden_nodes_layer1


10

In [20]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 =  int(round((hidden_nodes_layer1 + number_output_neurons)/2, 0))

# Review the number hidden nodes in the second layer
hidden_nodes_layer2


6

In [21]:
# Create the Sequential model instance
nn = Sequential()


In [22]:
# Add the first hidden layer
nn.add(Dense(units=hidden_nodes_layer1, activation="relu", input_dim=number_input_features))


In [23]:
# Add the second hidden layer
nn.add(Dense(units=hidden_nodes_layer2, activation="relu"))


In [24]:
# Add the output layer to the model specifying the number of output neurons and activation function
nn.add(Dense(units=number_output_neurons, activation="sigmoid"))


In [25]:
# Display the Sequential model summary
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                190       
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 66        
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 14        
Total params: 270
Trainable params: 270
Non-trainable params: 0
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [26]:
# Compile the Sequential model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [27]:
# Fit the model using 50 epochs and the training data
model = nn.fit(X_train_scaled, y_train, epochs=50, verbose=0)


### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [28]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=True)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

98/98 [==============================] - 0s 378us/step - loss: 0.1361 - accuracy: 0.9093
Loss: 0.13609716296195984, Accuracy: 0.9092658162117004


### Step 4: Save and export your model to an HDF5 file

In [29]:
# Set the model's file path
filename = "DLNN_model_1"
file_path = Path('Resources/' + filename + '.h5')

# Export your model to a HDF5 file
nn.save(file_path)


---

## Optimize the neural network model


### Step 1: Define at least three new deep neural network models (resulting in the original plus 3 optimization attempts). With each, try to improve on your first model’s predictive accuracy.

> **Rewind** Recall that perfect accuracy has a value of 1, so accuracy improves as its value moves closer to 1. To optimize your model for a predictive accuracy as close to 1 as possible, you can use any or all of the following techniques:
>
> * Adjust the input data by dropping different features columns to ensure that no variables or outliers confuse the model.
>
> * Add more neurons (nodes) to a hidden layer.
>
> * Add more hidden layers.
>
> * Use different activation functions for the hidden layers.
>
> * Add to or reduce the number of epochs in the training regimen.


### Alternative Model 1 - Deep learning model with 2 hidden layers

In [30]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])
print(number_input_features)


18


In [31]:
# Review the number of features
number_input_features

18

In [32]:
# Define the number of neurons in the output layer
number_output_neurons_A1 = 2

In [33]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A1 =  int(round((number_input_features + number_output_neurons_A1)/2, 0))
# Review the number hidden nodes in the first layer
hidden_nodes_layer1_A1


10

In [34]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2_A1 =  int(round((hidden_nodes_layer1_A1 + number_output_neurons_A1)/2, 0))

# Review the number hidden nodes in the second layer
hidden_nodes_layer2_A1


6

In [35]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer3_A1 =  int(round((hidden_nodes_layer2_A1 + number_output_neurons_A1)/2, 0))

# Review the number hidden nodes in the second layer
hidden_nodes_layer3_A1


4

In [36]:
# Create the Sequential model instance
nn_A1 = Sequential()

In [37]:
# Add the first hidden layer
nn_A1.add(Dense(units=hidden_nodes_layer1_A1, activation="relu", input_dim=number_input_features))


In [38]:
# Add the second hidden layer
nn_A1.add(Dense(units=hidden_nodes_layer2_A1, activation="relu"))


In [39]:
# Add the second hidden layer
nn_A1.add(Dense(units=hidden_nodes_layer3_A1, activation="relu"))


In [40]:
# Add the output layer to the model specifying the number of output neurons and activation function
nn_A1.add(Dense(units=number_output_neurons_A1, activation="sigmoid"))


In [41]:
# Display the Sequential model summary
nn_A1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 10)                190       
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 66        
_________________________________________________________________
dense_5 (Dense)              (None, 4)                 28        
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 10        
Total params: 294
Trainable params: 294
Non-trainable params: 0
_________________________________________________________________


In [42]:
# Compile the Sequential model
nn_A1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [43]:
# Fit the model using 50 epochs and the training data
model_A1 = nn_A1.fit(X_train_scaled, y_train, epochs=50, verbose=0)


### Alternative Model 2 - Deep learning model with 3 hidden layers

In [44]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])
print(number_input_features)


18


In [45]:
# Review the number of features
number_input_features

18

In [46]:
# Define the number of neurons in the output layer
number_output_neurons_A2 = 2

In [47]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A2 =  int(round((number_input_features + number_output_neurons_A2)/2, 0))
# Review the number hidden nodes in the first layer
hidden_nodes_layer1_A2


10

In [48]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2_A2 =  int(round((hidden_nodes_layer1_A2 + number_output_neurons_A2)/2, 0))

# Review the number hidden nodes in the second layer
hidden_nodes_layer2_A2


6

In [49]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer3_A2 =  int(round((hidden_nodes_layer2_A1 + number_output_neurons_A2)/2, 0))

# Review the number hidden nodes in the second layer
hidden_nodes_layer3_A2


4

In [50]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer4_A2 =  int(round((hidden_nodes_layer3_A1 + number_output_neurons_A2)/2, 0))

# Review the number hidden nodes in the second layer
hidden_nodes_layer4_A2


3

In [51]:
# Create the Sequential model instance
nn_A2 = Sequential()

In [52]:
# Add the first hidden layer
nn_A2.add(Dense(units=hidden_nodes_layer1_A2, activation="relu", input_dim=number_input_features))


In [53]:
# Add the second hidden layer
nn_A2.add(Dense(units=hidden_nodes_layer2_A2, activation="relu"))


In [54]:
# Add the second hidden layer
nn_A2.add(Dense(units=hidden_nodes_layer3_A2, activation="relu"))


In [55]:
# Add the second hidden layer
nn_A2.add(Dense(units=hidden_nodes_layer4_A2, activation="relu"))


In [56]:
# Add the output layer to the model specifying the number of output neurons and activation function
nn_A2.add(Dense(units=number_output_neurons_A2, activation="sigmoid"))


In [57]:
# Display the Sequential model summary
nn_A2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 10)                190       
_________________________________________________________________
dense_8 (Dense)              (None, 6)                 66        
_________________________________________________________________
dense_9 (Dense)              (None, 4)                 28        
_________________________________________________________________
dense_10 (Dense)             (None, 3)                 15        
_________________________________________________________________
dense_11 (Dense)             (None, 2)                 8         
Total params: 307
Trainable params: 307
Non-trainable params: 0
_________________________________________________________________


In [58]:
# Compile the Sequential model
nn_A2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [59]:
# Fit the model using 50 epochs and the training data
model_A2 = nn_A2.fit(X_train_scaled, y_train, epochs=50, verbose=0)


#### Alternative Model 3 - Principal Component Analysis

In [60]:
from sklearn.decomposition import PCA


In [61]:
# Create a StandardScaler instance
pca_scaler = StandardScaler()

# Fit the scaler to the features
X_pca_scaler = pca_scaler.fit(X)

# Fit the scaler to the features
X_pca_scaled = X_pca_scaler.transform(X)


In [62]:
for n in range(2, number_input_features):
    pca = PCA(n_components=n, random_state=1)
    X_pca = pca.fit_transform(X_pca_scaled)
    exp_var = pca.explained_variance_ratio_
#     print(f'{exp_var}')
    print(f'N = {n:3}: Total explained variance: {100*exp_var.sum():.4f}%')

N =   2: Total explained variance: 41.8793%
N =   3: Total explained variance: 49.4176%
N =   4: Total explained variance: 56.1540%
N =   5: Total explained variance: 62.2239%
N =   6: Total explained variance: 68.1662%
N =   7: Total explained variance: 74.0366%
N =   8: Total explained variance: 79.8494%
N =   9: Total explained variance: 85.1969%
N =  10: Total explained variance: 90.2461%
N =  11: Total explained variance: 94.4264%
N =  12: Total explained variance: 97.7982%
N =  13: Total explained variance: 99.9872%
N =  14: Total explained variance: 99.9980%
N =  15: Total explained variance: 99.9992%
N =  16: Total explained variance: 99.9998%
N =  17: Total explained variance: 100.0000%


In [63]:
best_n = 13
pca = PCA(n_components=best_n, random_state=1)
X_pca = pca.fit_transform(X_pca_scaled)
exp_var = pca.explained_variance_ratio_
print(f'N = {best_n:3}: Total explained variance: {100*exp_var.sum():.4f}%')


N =  13: Total explained variance: 99.9872%


In [64]:
X_pca_train, X_pca_test, y_pca_train, y_pca_test = train_test_split(X_pca, y, random_state=1)

In [65]:
# Define the the number of inputs (features) to the model
number_input_features = best_n
print(number_input_features)


13


In [66]:
# Define the number of neurons in the output layer
number_output_neurons_A3 = 2

In [67]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A3 =  int(round((number_input_features + number_output_neurons_A3)/2, 0))
# Review the number hidden nodes in the first layer
hidden_nodes_layer1_A3


8

In [68]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2_A3 =  int(round((hidden_nodes_layer1_A3 + number_output_neurons_A3)/2, 0))

# Review the number hidden nodes in the second layer
hidden_nodes_layer2_A3


5

In [69]:
# Define the number of hidden nodes for the third hidden layer
hidden_nodes_layer3_A3 =  int(round((hidden_nodes_layer2_A3 + number_output_neurons_A3)/2, 0))

# Review the number hidden nodes in the third layer
hidden_nodes_layer3_A3


4

In [70]:
# Create the Sequential model instance
nn_A3 = Sequential()

In [71]:
# Add the first hidden layer
nn_A3.add(Dense(units=hidden_nodes_layer1_A3, activation="relu", input_dim=number_input_features))


In [72]:
# Add the second hidden layer
nn_A3.add(Dense(units=hidden_nodes_layer2_A3, activation="relu"))


In [73]:
# Add the third hidden layer
nn_A3.add(Dense(units=hidden_nodes_layer3_A3, activation="relu"))


In [74]:
# Add the output layer to the model specifying the number of output neurons and activation function
nn_A3.add(Dense(units=number_output_neurons_A3, activation="sigmoid"))


In [75]:
# Display the Sequential model summary
nn_A3.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 8)                 112       
_________________________________________________________________
dense_13 (Dense)             (None, 5)                 45        
_________________________________________________________________
dense_14 (Dense)             (None, 4)                 24        
_________________________________________________________________
dense_15 (Dense)             (None, 2)                 10        
Total params: 191
Trainable params: 191
Non-trainable params: 0
_________________________________________________________________


In [76]:
# Compile the Sequential model
nn_A3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [77]:
# Fit the model using 50 epochs and the training data
model_A3 = nn_A3.fit(X_pca_train, y_pca_train, epochs=50, verbose=1)


Epoch 1/50
293/293 [==============================] - 0s 456us/step - loss: 0.4515 - accuracy: 0.8421
Epoch 2/50
293/293 [==============================] - 0s 454us/step - loss: 0.1906 - accuracy: 0.9043
Epoch 3/50
293/293 [==============================] - 0s 451us/step - loss: 0.1698 - accuracy: 0.9075
Epoch 4/50
293/293 [==============================] - 0s 444us/step - loss: 0.1610 - accuracy: 0.9135
Epoch 5/50
293/293 [==============================] - 0s 451us/step - loss: 0.1561 - accuracy: 0.9172
Epoch 6/50
293/293 [==============================] - 0s 454us/step - loss: 0.1533 - accuracy: 0.9166
Epoch 7/50
293/293 [==============================] - 0s 458us/step - loss: 0.1513 - accuracy: 0.9157
Epoch 8/50
293/293 [==============================] - 0s 454us/step - loss: 0.1497 - accuracy: 0.9159
Epoch 9/50
293/293 [==============================] - 0s 458us/step - loss: 0.1483 - accuracy: 0.9147
Epoch 10/50
293/293 [==============================] - 0s 454us/step - loss: 0.147

### Step 2: After finishing your models, display the accuracy scores achieved by each model, and compare the results.

In [78]:
print("Original Model Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=True)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Original Model Results
98/98 [==============================] - 0s 395us/step - loss: 0.1361 - accuracy: 0.9093
Loss: 0.13609716296195984, Accuracy: 0.9092658162117004


In [79]:
print("Alternative Model 1 Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_A1.evaluate(X_test_scaled, y_test, verbose=True)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Alternative Model 1 Results
98/98 [==============================] - 0s 401us/step - loss: 0.1396 - accuracy: 0.9093
Loss: 0.13957563042640686, Accuracy: 0.9092658162117004


In [80]:
print("Alternative Model 2 Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_A2.evaluate(X_test_scaled, y_test, verbose=True)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Alternative Model 2 Results
98/98 [==============================] - 0s 385us/step - loss: 0.1541 - accuracy: 0.9089
Loss: 0.15406441688537598, Accuracy: 0.9089452028274536


In [81]:
print("Alternative Model 3 Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_A3.evaluate(X_pca_test, y_pca_test, verbose=True)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Alternative Model 3 Results
98/98 [==============================] - 0s 381us/step - loss: 0.1372 - accuracy: 0.9093
Loss: 0.1371750682592392, Accuracy: 0.9092658162117004


### Run different numbers of epochs on best model

In [82]:
# Fit the model using the training data with different numbers of epochs
model_list = []
for n in range(50, 151, 20):
    print(f'Fitting model, epochs: {n}')
    model_list.append(nn_A1.fit(X_train_scaled, y_train, epochs=50, verbose=False))
#     model_A1 = nn_A1.fit(X_train_scaled, y_train, epochs=50, verbose=False)

    # Evaluate the model loss and accuracy metrics using the evaluate method and the test data
    model_loss, model_accuracy = nn_A1.evaluate(X_test_scaled, y_test, verbose=True)
    # Display the model loss and accuracy results
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")    

Fitting model, epochs: 50
98/98 [==============================] - 0s 401us/step - loss: 0.1378 - accuracy: 0.9093
Loss: 0.1377825289964676, Accuracy: 0.9092658162117004
Fitting model, epochs: 70
98/98 [==============================] - 0s 380us/step - loss: 0.1382 - accuracy: 0.9096
Loss: 0.1382400095462799, Accuracy: 0.9095864295959473
Fitting model, epochs: 90
98/98 [==============================] - 0s 412us/step - loss: 0.1395 - accuracy: 0.9096
Loss: 0.13950130343437195, Accuracy: 0.9095864295959473
Fitting model, epochs: 110
98/98 [==============================] - 0s 401us/step - loss: 0.1434 - accuracy: 0.9093
Loss: 0.14337988197803497, Accuracy: 0.9092658162117004
Fitting model, epochs: 130
98/98 [==============================] - 0s 411us/step - loss: 0.1460 - accuracy: 0.9093
Loss: 0.14596973359584808, Accuracy: 0.9092658162117004
Fitting model, epochs: 150
98/98 [==============================] - 0s 401us/step - loss: 0.1542 - accuracy: 0.9083
Loss: 0.15420414507389069, Ac

In [83]:
# for model in model_list:
#     pass
#     print(model.history)
    # Evaluate the model loss and accuracy metrics using the evaluate method and the test data
#     model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=True)
    # Display the model loss and accuracy results
#     print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")    

### Step 3: Save each of your alternative models as an HDF5 file.


In [84]:
# Set the file path for the first alternative model
filename = "DLNN_model_A1"
file_path_A1 = Path('Resources/' + filename + '.h5')

# Export your model to a HDF5 file
nn_A1.save(file_path_A1)


In [85]:
# Set the file path for the second alternative model
filename = "DLNN_model_A2"
file_path_A2 = Path('Resources/' + filename + '.h5')

# Export your model to a HDF5 file
nn_A2.save(file_path_A2)


In [86]:
# # Set the file path for the third alternative model
filename = "DLNN_model_A3"
file_path_A3 = Path('Resources/' + filename + '.h5')

# # Export your model to a HDF5 file
nn_A3.save(file_path_A3)
